In [1]:
###TASK 1 - LINKS
import pandas as pd
# vf_stats = pd.read_csv('link_stats_jun20.csv')
lf_meta = pd.read_csv('../models-links/radshift_links_2016_2017.csv')

all_pages = lf_meta['page'].unique()

###which page is best fit for a given just the title - 
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
import numpy as np

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []

cv_folds = 10
results = pd.DataFrame()
page_clf = []

df = lf_meta[['title', 'page']].dropna()

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.page == test_page]
    X_0 = df[~(df.page == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['title', 'page']]
    X_0 = X_0[['title', 'page']]
    X_all = pd.concat([X_1, X_0])
    
#     print(len(X_1), ":", len(X_0))
    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['page'].apply(get_class, page = test_page)
#     X_all = X_all.dropna()
    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
#                          ('featselect', SelectKBest(k = 100)),
                         ('clf', MultinomialNB()),
#                          ('clf', LinearSVC()),
#                          ('clf', RandomForestClassifier()),
#                          ('clf', LogisticRegression()),
    ])
    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['title']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred, average='macro')
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
#     print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

results.mean()

### First page promotion
###Total data points: 85591
# num_pages: 85
# num_links    1448.000000
# accuracy        0.789186
# precision       0.811785
# recall          0.792267
# fscore          0.778326

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


num_links    1448.000000
accuracy        0.794669
precision       0.818950
recall          0.796943
fscore          0.784692
dtype: float64

In [80]:

#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed Quiz"
# q_page = "BuzzFeed Japan"

q_test = ["5 DIY Soap Ideas"]
# q_test = ['Camping Meal Prep Hacks']
# q_test = ["People Try Kardashian Beauty Secrets"]
# q_test = ['This Restaurant In Mumbai Is Serving Up Goth Tikka']  ###list of titles you wanna query
# q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

print("Finding best first page for item titled: ", q_test)

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
#         print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page, "; prob = ", max_prob)

Finding best first page for item titled:  ['5 DIY Soap Ideas']
BuzzFeed Rewind 0.265007705803
BuzzFeed UK 0.284178758104
BuzzFeed 0.45175979813
BuzzFeed Food 0.637416173533
BuzzFeed Entertainment 0.224044450071
BuzzFeed Books 0.469000600401
BuzzFeed Science 0.626617163446
BuzzFeed Parents 0.739469444811
Tasty 0.741292059667
BuzzFeed Australia 0.430585996526
BuzzFeed Animals 0.376352166902
BuzzFeed UK Politics 0.359885671701
BuzzFeed Quiz 0.231197054823
BuzzFeed France 0.260836904284
BuzzFeed DIY 0.918669832635
BuzzFeed Brasil 0.172660611243
Tasty Demais 0.493506493506
BuzzFeed Español 0.630073431321
BuzzFeed Philippines 0.443954486702
BuzzFeed Health 0.463773525657
Buy Me That 0.753334275638
BuzzFeed India 0.533303007184
BuzzFeed Ladylike 0.400709939303
BuzzFeed Community 0.434832074099
BuzzFeed Deutschland 0.345362346837
SOML 0.365831883563
BuzzFeed Scotland 0.486842105263
BuzzFeed News 0.186651388651
Quizzes En Español 0.291019055028
BuzzFeed Video 0.4753073691
Bien Tasty 0.508771929

In [81]:
test_page = 'Tasty'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:10]['title']:
    print(nam)

Top 10 links that are most relevant for FB  Tasty
--------------
You Can Only Eat Pasta If You Get 12/15 In This Quiz
7 Dinners Under $10 You Should Make This Week
Only A True Pizza Expert Can Identify Which Restaurant These Pizzas Are From Based On A Picture
7 Small Ways To Actually Get Your Home In Order
How Much Do You Actually Know About Cooking Eggs?
23 Cheap Products That'll Actually Keep Your Kitchen Organized
We Know Which Job You Should Actually Have Based On Your Apartment Preferences
Pick Seven Of Your Favorite Foods And We'll Reveal What Your Life Will Be Like In 10 Years
Your Proposal Preferences Will Reveal When You'll Get Engaged
Tell Us Eight Of Your Food Preferences And We'll Reveal Where You Should Move To


In [82]:
test_page = 'Nifty'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:10]['title']:
    print(nam)

Top 10 links that are most relevant for FB  Nifty
--------------
23 Easy Hacks That'll Make Every Lazy Girl's Life So Much Easier
24 Ways To Decorate Your Home Without Making It Look Cluttered
23 YouTube Couples That Will Make You Scream &quot;Relationship Goals!&quot;
23 Cheap Products That'll Actually Keep Your Kitchen Organized
21 Ridiculously Clever Mugs That Are 100% Funnier Than You
333 Weight Loss Tips That You'll Wish You Read Sooner
26 Splurge-Worthy Fitness Products That Are Actually Worth Your Money
19 Dirty Tweets That Will Make You Laugh Way Harder Than You Should
23 Books Guaranteed To Give You That Summer Feeling
How Much Do You Actually Know About Cooking Eggs?


In [71]:
test_page = 'BuzzFeed Animals'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
# temp = lf_meta[['title', 'page']]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:20]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed Animals
--------------
People Are Sharing Their Dogs' Adorable Transformations From Puppies To Doggos
17 Tweets About Animals That Will Make Your Eyes Swell Up With Tears Of Joy
15 Tweets That Are Way Too Relatable To People Who Don't Smile
19 Dirty Tweets That Will Make You Laugh Way Harder Than You Should
Pick Sweet Or Cheesy
Honestly Leo, Who Does That?!
Literally Just 17 Hilarious Tweets From People That'll Make You Pee Your Pants
18 Tumblr Posts You'll Want To Send To Your Best Friend Immediately
The RompDad Is So Pure It Will Restore Your Faith In Humanity
11 Stupid-Ass Decisions That Prove The Censor Board Is A Bunch Of Class-A Morons
Can You Guess The Dog Breed From The Crappy Drawing?
This Woman Captured The Adorable Moment Her UPS Driver Took Selfies With Her Dogs
21 Funny Tweets Aimed Specifically At Anyone Who’s Ever Had A Period
18 People Who Are Way Too Smart For Their Own Good
22 Things That Will Make Anyone Who's Been

In [72]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Rewind,2106,0.769468,0.804743,0.770652,0.762497
1,BuzzFeed UK,4096,0.670780,0.704175,0.671220,0.656821
2,BuzzFeed,10435,0.617729,0.663590,0.617901,0.588998
3,BuzzFeed Food,2855,0.815061,0.837122,0.815317,0.811916
4,BuzzFeed Entertainment,2791,0.792731,0.807137,0.792996,0.790220
5,BuzzFeed Books,1978,0.747216,0.783619,0.747919,0.738632
6,BuzzFeed Science,269,0.778826,0.781032,0.781168,0.776306
7,BuzzFeed Parents,940,0.703191,0.764443,0.703677,0.684662
8,Tasty,972,0.774716,0.813309,0.774685,0.766898
9,BuzzFeed Australia,5360,0.641978,0.679455,0.642291,0.622433


In [73]:
test_page = 'BuzzFeed Style'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
# temp = lf_meta[['title', 'page']]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]
print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:20]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed Style
--------------
All The Best Deals On Amazon Today
All The Best Deals On The Internet This Weekend
29 Of The Best Beauty Products To Buy At Target
29 Gorgeous And Cheap Dresses To Wear To An Outdoor Wedding
How Would You Wear These Red High Heels?
11 Beauty Products That Will Actually Take Off All Your Makeup
25 Gorgeous Nail Polishes People Actually Swear By
26 Awesome Beauty Products You Didn't Know You Could Get At Urban Outfitters
26 Of The Best Places To Buy Last Minute Gifts
Here's What Amazon Prom Dresses Look Like On Non-Models
Here's What Forever 21's New Bridesmaid Dresses Look Like IRL
Create A Sitcom And We'll Reveal Your Hair And Eye Color
8 Life-Changing Tech Products You Need ASAP
19 Photos That Prove Natural Hair Doesn't Need Twist-Outs To Look Beautiful
21 Beautiful Ring Sets You'll Want On Your Fingers ASAP
23 Easy Hacks That'll Make Every Lazy Girl's Life So Much Easier
Kim Kardashian Is Launching Her Own Make

In [75]:
test_page = 'BuzzFeed News'
ind = pages.index(test_page)
clf = page_clf[ind]

temp = lf_meta[~(lf_meta.page == test_page)]
# temp = lf_meta[~(lf_meta.page == test_page)]
temp = temp.sort_values('stats_date', ascending = False)[:1000]
# temp = lf_meta[['title', 'page']]
query_titles = temp['title'].unique()

def get_title_fit(X, clf):
     return clf.predict_proba(X)

demo = pd.DataFrame()
demo['title'] = query_titles
temp = pd.DataFrame(clf.predict_proba(query_titles), columns=clf.classes_)
demo['prob'] = temp[1]

print("Top 10 links that are most relevant for FB ", test_page)
print("--------------")
for nam in demo.sort_values('prob', ascending = False)[:20]['title']:
    print(nam)

Top 10 links that are most relevant for FB  BuzzFeed News
--------------
Police Officer Found Not Guilty In The Fatal Shooting Of Philando Castile
Live Updates: Finsbury Park Terror Suspect Remains In Custody As Police Obtain Warrant For Further Detention
Six People Have Resigned From Trump's HIV/AIDS Advisory Council Because He &quot;Doesn't Care&quot;
Kochi Metro Normalises Trans People At Work With This Message To Passengers
An Aussie Bloke Allegedly Tunnelled His Way Out Of A Bali Jail Just Months Before His Release Date
Doctors Dispute North Korea's Reason For US Student Being Comatose
Vice President Pence Has Hired His Own Lawyer In Connection With The Special Counsel Probe
New Zealand Has The Highest Adolescent Suicide Rate Of All Wealthy Countries, Says New Report
Trump Rolls Back Obama’s Cuba Policies On Travel And Trade
Steve Harvey Gets Roasted On Twitter After Tasteless Flint Water Crisis Joke
This Couple Turned 42 On The Same Day And Did The Cutest Birthday Photo Shoot
Whe

In [66]:
temp = lf_meta[~(lf_meta.page == test_page)]
temp[temp.page == 'BuzzFeed News']

,external_id,created_at,page,post_type,buzz_id,uri,title,stats_ts,stats_date,consumptions_by_type__link_clicks,...,stories_by_action_type__comment,stories_by_action_type__like,stories_by_action_type__share,engaged_fan,fan_reach,impressions,impressions_fan,twoday_stats_date,twoday_consumptions_by_type__link_clicks,twoday_stories_by_action_type__share


In [ ]:
Build A Wardrobe From Forever 21 And We will Guess Your Signature Style;Which Game Of Thrones Character Are You Based On Your Taste In Food?;18 GIFs That Sum Up Your Relationship With Summer 